# Managing population growth

In [159]:
import sys
import numpy as np
import pandas as pd
import altair as alt
sys.path.append('..')
import helpers.dbquery as db
import helpers.functions as f
import yaml, ssl, asyncio, pickle
import nb_black

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 
nest_asyncio.apply()

In [160]:
# mapping to the modules that make the app
sys.path.insert(0, "../../app")
import models

c = models.CosmosdbClient()

In [161]:
params = yaml.safe_load(open('../../functions/popupdate/settings.yaml'))
syllables = pickle.load(open(os.path.join(os.getenv("abspath"),"app/creators/specs/syllables.p"), "rb"))
username = 'notebook'
params

{'pop_health_requirement': 0.3}

In [162]:
healthy_pops_query = f"""
g.V().has('label','pop')
    .has('health',gt({params['pop_health_requirement']})).as('pop')
    .local(
        union(
            out('enhabits').as('location'),
            out('isOfSpecies').as('species')
            )
		    .fold()).as('location','species')
	    .path()
		.by(unfold().valueMap().fold())
"""
c.run_query(healthy_pops_query)
data = c.reduce_res(c.res)
print("Total pops loaded: ",len([d['pop'] for d in data]))

Total pops loaded:  51


## Population Growth

In [163]:
pops_df = pd.DataFrame([d['pop'] for d in data])
pops_df.head()

,health,name,objid,conformity,literacy,aggression,constitution,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.5,Yo,3667258216475,0.110,0.723,0.441,0.563,3073058128292,0.5020,0.6125,0.383,True,Billmanserver,pop,3667258216475
1,0.5,Guagrasauhra,8547880276199,0.209,0.494,0.638,0.373,3073058128292,0.5055,0.4998,0.223,True,Billmanserver,pop,8547880276199
2,0.5,Ly,4752277769726,0.053,0.480,0.769,0.708,3073058128292,0.7385,0.6092,0.141,True,Billmanserver,pop,4752277769726
3,0.5,Tamkobadsun Benbalhra,4557665522436,0.318,0.574,0.810,0.685,1327778884546,0.7475,0.6607,0.681,True,Billmanserver,pop,4557665522436
4,0.5,Kiyley,1043563010918,0.076,0.583,0.170,0.812,2905465436322,0.4910,0.5370,0.100,True,Billmanserver,pop,1043563010918


In [215]:
species_df = pd.DataFrame([d['species'] for d in data])
locations_df = pd.DataFrame([d['location'] for d in data])

locations_df.head()

,name,class,objid,radius,mass,orbitsDistance,orbitsId,orbitsName,isSupportsLife,isPopulated,type,isHomeworld,username,objtype,id
0,Batsambi,terrestrial,5960500473740,0.4377,0.8065,1,1146706061470,Heights,True,True,terrestrial,True,Billmanserver,planet,5960500473740
1,Batsambi,terrestrial,5960500473740,0.4377,0.8065,1,1146706061470,Heights,True,True,terrestrial,True,Billmanserver,planet,5960500473740
2,Batsambi,terrestrial,5960500473740,0.4377,0.8065,1,1146706061470,Heights,True,True,terrestrial,True,Billmanserver,planet,5960500473740
3,Batsambi,terrestrial,5960500473740,0.4377,0.8065,1,1146706061470,Heights,True,True,terrestrial,True,Billmanserver,planet,5960500473740
4,Batsambi,terrestrial,5960500473740,0.4377,0.8065,1,1146706061470,Heights,True,True,terrestrial,True,Billmanserver,planet,5960500473740


In [164]:
pops_df['roll'] = pops_df['objid'].apply(lambda x: np.random.random())
pops_df['grow'] = pops_df[['wealth','health']].T.mean() >= pops_df['roll']
pops_df.groupby('grow')['id'].count()

grow
False    24
True     27
Name: id, dtype: int64

In [225]:
reproducing_pops = pops_df[pops_df['grow']].drop(['roll','grow'],axis=1)
reproducing_pops.head()

,health,name,objid,conformity,literacy,aggression,constitution,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.5,Yo,3667258216475,0.110,0.723,0.441,0.563,3073058128292,0.5020,0.6125,0.383,True,Billmanserver,pop,3667258216475
3,0.5,Tamkobadsun Benbalhra,4557665522436,0.318,0.574,0.810,0.685,1327778884546,0.7475,0.6607,0.681,True,Billmanserver,pop,4557665522436
4,0.5,Kiyley,1043563010918,0.076,0.583,0.170,0.812,2905465436322,0.4910,0.5370,0.100,True,Billmanserver,pop,1043563010918
9,0.5,Y,3104591820503,0.196,0.508,0.605,0.534,3073058128292,0.5695,0.5388,0.054,True,Billmanserver,pop,3104591820503
10,0.5,,6236201872414,0.441,0.398,0.440,0.563,2905465436322,0.5015,0.4498,0.492,True,Billmanserver,pop,6236201872414


The AZ-functions is a different build than the full website. So, I need to separate a number of functions that I need to run in the AZ functions box.

In [185]:
def uuid(n=13):
    return "".join([str(i) for i in np.random.choice(range(10), n)])

def make_word(n):
    syl = np.random.choice(syllables, n)
    word = "".join(syl)
    return word.capitalize()


In [227]:
changing_values = ['conformity','literacy','aggression','constitution','wealth','factionLoyalty']


def grow_pop(p,species):
    child = p.copy()
    child['name'] = child['name']+make_word(1).lower()
    id = uuid()
    child['objid'] = id
    child['id'] = id
    child['isIdle'] = 'False'
    child['health'] = np.round(child['health']*.6,3)
    child['wealth'] = np.round(child['wealth']*.6,3)
    child['industry'] = np.round(child['industry']*.6,3)
    for v in changing_values:
        child[v] = np.round(child[v] + np.random.uniform(low=-.1, high=.1),3)
    return child




In [221]:
data[0]['pop']

{'health': 0.5,
 'name': ' Yo',
 'objid': '3667258216475',
 'conformity': 0.11,
 'literacy': 0.723,
 'aggression': 0.441,
 'constitution': 0.563,
 'isInFaction': '3073058128292',
 'industry': 0.502,
 'wealth': 0.6125,
 'factionLoyalty': 0.383,
 'isIdle': 'True',
 'username': 'Billmanserver',
 'objtype': 'pop',
 'id': '3667258216475'}

In [222]:
grow_pop(data[0]['pop'],data[0]['species'])

{'health': 0.3,
 'name': ' Yony',
 'objid': '0445829141765',
 'conformity': 0.206,
 'literacy': 0.801,
 'aggression': 0.46,
 'constitution': 0.639,
 'isInFaction': '3073058128292',
 'industry': 0.301,
 'wealth': 0.376,
 'factionLoyalty': 0.3,
 'isIdle': 'False',
 'username': 'Billmanserver',
 'objtype': 'pop',
 'id': '0445829141765',
 'childOf': '3667258216475'}

Getting the nodes and edges for that population:

In [232]:
nodes = []
edges = []

for i in reproducing_pops.index.to_list():
    p = reproducing_pops.loc[i].to_dict()
    species = species_df.loc[i].to_dict()
    child = grow_pop(p,species)
    nodes.append(child)
    edges.append({"node1": child["objid"], "node2": p["objid"], "label": "childOf"})
    edges.append({"node1": child["objid"], "node2": child["isInFaction"], "label": "isInFaction"})
    edges.append({"node1": child["objid"], "node2": species["objid"], "label": "isOfSpecies"})
    

In [233]:
pd.DataFrame(nodes)

,health,name,objid,conformity,literacy,aggression,constitution,isInFaction,industry,wealth,factionLoyalty,isIdle,username,objtype,id
0,0.3,Yono,5092982627872,0.088,0.701,0.441,0.526,3073058128292,0.301,0.428,0.474,False,Billmanserver,pop,5092982627872
1,0.3,Tamkobadsun Benbalhraje,8131231289025,0.350,0.555,0.724,0.759,1327778884546,0.448,0.462,0.604,False,Billmanserver,pop,8131231289025
2,0.3,Kiyleybo,6778029734859,0.109,0.537,0.160,0.754,2905465436322,0.295,0.405,0.132,False,Billmanserver,pop,6778029734859
3,0.3,Yhas,0615156744235,0.239,0.596,0.549,0.502,3073058128292,0.342,0.354,0.067,False,Billmanserver,pop,0615156744235
4,0.3,bel,7124429303659,0.377,0.449,0.539,0.548,2905465436322,0.301,0.331,0.575,False,Billmanserver,pop,7124429303659
5,0.3,Dosbridconcheguaju,2444554190736,0.031,0.698,0.795,0.484,3073058128292,0.350,0.458,0.172,False,Billmanserver,pop,2444554190736
6,0.3,Tamkobadsun fo,8330399758760,0.659,0.589,0.607,0.867,1327778884546,0.442,0.465,0.646,False,Billmanserver,pop,8330399758760
7,0.3,Ren Badchenueburgpur,5503671084684,0.726,0.798,0.477,0.527,6035778500432,0.312,0.352,0.583,False,william.jeffrey.harding@gmail.com,pop,5503671084684
8,0.3,Ren Rezvankun,7105003329875,0.530,0.483,0.495,0.478,6035778500432,0.281,0.264,0.521,False,william.jeffrey.harding@gmail.com,pop,7105003329875
9,0.3,Pecmun Jarram,9956968365070,0.304,0.323,0.520,0.568,8012031023111,0.359,0.202,0.386,False,william.jeffrey.harding@gmail.com,pop,9956968365070


In [234]:
pd.DataFrame(edges)

,node1,node2,label
0,5092982627872,3667258216475,childOf
1,5092982627872,3073058128292,isInFaction
2,5092982627872,2331376965323,isOfSpecies
3,8131231289025,4557665522436,childOf
4,8131231289025,1327778884546,isInFaction
...,...,...,...
76,6356351673169,5791656630325,isInFaction
77,6356351673169,4734174670414,isOfSpecies
78,1739624312445,7690193253130,childOf
79,1739624312445,5791656630325,isInFaction
